# Implementierung des Gauss Algorithmus

## inputs, config and utils:

In [331]:
import numpy as np
from functools import reduce

DO_PIVOT = True
PRINT_INTERMEDIATE_RESULTS = True

In [332]:
# we assume the permutations p1, p2 are over the same space 1,....,n
# the i-th entry of an permutation p tells us on which position the i-th row of the matrix sits after the permutation
# example: when n=3 is p1 = [0,1,2] the identity permutation and [2,1,0] switches rows 0 and 2
# we can use reduce in combination with concat_permutation to concat a array of permutations
def concat_permutations(p2, p1):
    result = []
    for i in range(0, len(p1)):
        result.append(p2[p1[i]])
    
    return result

def decode_permutation(p):
# to performe the permutation we will decode p into the permutation matrix and perform matrix multiplication
    n = len(p)
    p_ = np.matrix(np.zeros((n, n), dtype=int))

    for i in range(0,n):
        p_[i,p[i]] = 1
    
    return p_
    
def permutate_matrix_left(p, a):
    return decode_permutation(p)*a

def permutate_matrix_right(a,p):
    return a*decode_permutation(p)

def permutate_matrix(p,a):
    return permutate_matrix_left(p, permutate_matrix_right(a,p))

In [333]:
n = 3

#a = [[1,-2,3],[3,-5,7],[-2,2,-7]] #the matrix is an array of columns
a = np.matrix([[1,3,-2],[-2,-5,2],[3,7,-7]])
a_original = np.matrix([[1,3,-2],[-2,-5,2],[3,7,-7]])
#b = [2,-3,9]
b = np.matrix([[2],[-3],[9]])

# list of all f and p matricies we create in each step, this is needed to compute l in the end
fs = []
ps= []

## 1. Step: Berechnung der Matrizen P,L,R 

Calculate r and the permutation and elimination matrix for each column

In [334]:
# loop through every column (every inner array)
for i in range(0,n-1):
    # we are inside column i
    if (DO_PIVOT):
        pivot_value = a[i,i]
        pivot_position = i

        # interpret p as a permutation, 
        p = list(range(0,n))
        for j in range(i+1, n):
            if abs(a[j,i]) > abs(pivot_value):
                # if we have a bigger element change the pivot element and position
                pivot_value = a[j,i]
                pivot_position = j
                
        if (pivot_position != i):
            # if we have found a new pivot element switch the rows according to p
            safe = p[i]
            p[i] = p[pivot_position]
            p[pivot_position] = safe
            a = permutate_matrix_left(p, a)
        ps.append(p)
    
    # after we changed the necessary rows and updated our p we can calculate the elimination matrix
    # create a new elimination matrix f_i
    f = np.matrix(np.eye(n))
    for k in range(i+1, n):
        f[k,i] = -(a[k,i] / a[i,i])

    # after we have caluclated the elimination matrix we can calculate the next a
    fs.append(f)
    a = f* a

r = a
print(r)

[[ 3.          7.         -7.        ]
 [ 0.          0.66666667  0.33333333]
 [ 0.          0.         -2.5       ]]


## compute p and l

In [335]:
# use reduce to concat all the ps to p
if DO_PIVOT:
    p = reduce(lambda x, y: concat_permutations(x,y), ps)

fs_hat = []
# iterate through fs in reverse order
for i in range(n-2, -1, -1):
    f = fs[i]
    for j in range(i+1, n-1):
        f = permutate_matrix(ps[j],f)
    fs_hat.append(f)

# the first element of fs_hat is ^f_n-2
l = np.matrix(np.eye(n))
for i in range(0,n-1):
    l = fs_hat[i].getI() * l

In [336]:
if PRINT_INTERMEDIATE_RESULTS:
    print("P:")
    print(p)
    print("\n")

    print("R:")
    print(r)
    print("\n")

    print("L:")
    print(l)
    print("\n")

    print("PA:")
    print(permutate_matrix_left(p, a_original))
    print("\n")
    
    print("LR:")
    print(l*r)
    print("\n")

P:
[2, 0, 1]


R:
[[ 3.          7.         -7.        ]
 [ 0.          0.66666667  0.33333333]
 [ 0.          0.         -2.5       ]]


L:
[[ 1.          0.          0.        ]
 [ 0.33333333  1.          0.        ]
 [-0.66666667 -0.5         1.        ]]


PA:
[[ 3  7 -7]
 [ 1  3 -2]
 [-2 -5  2]]


LR:
[[ 3.  7. -7.]
 [ 1.  3. -2.]
 [-2. -5.  2.]]




## 2. Loesen des Systems Lc = Pb

In [337]:
# define c := Rx = [x'_0, ..... , x'_n-1]

b_new = permutate_matrix_left(p, b)

c = np.matrix(np.zeros((n, 1)))

# use forward substitution to solve lc = b_new
for i in range(0,n):
    matrix_row_c = 0
    for j in range(0, i):
        matrix_row_c += l[i,j] * c[j]

    c[i] = (b_new[i] - matrix_row_c) / l[i,i]



In [338]:
if PRINT_INTERMEDIATE_RESULTS:
    print("Pb:")
    print(b_new)
    print("\n")

    print("c:")
    print(c)
    print("\n")

Pb:
[[ 9]
 [ 2]
 [-3]]


c:
[[ 9. ]
 [-1. ]
 [ 2.5]]




## 3. Loesen des Systems Rx = c

In [339]:
# with the computed c we can use rx = c using backward substitution

x = np.matrix(np.zeros((n, 1)))

for i in range(n-1,-1,-1):
    matrix_row_x = 0
    for j in range(i+1,n):
        matrix_row_x += r[i,j] * x[j]
    x[i] = (c[i] - matrix_row_x) / r[i,i]

Resultat:

In [340]:
print(x)

[[ 3.]
 [-1.]
 [-1.]]
